# Demo de Cadena de Markov para completar texto

---


Se base en ejempo propuesto en https://towardsdatascience.com/text-generation-with-markov-chains-an-introduction-to-using-markovify-742e6680dc33 usando https://github.com/jsvine/markovify

In [1]:
#@title Instalar librería Markovify

!pip install markovify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#@title Librerías a usar

##import spacy
import re
import markovify

import tensorflow as tf
import unicodedata
import copy
import random

from IPython.display import Image, display

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import math

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


import ipywidgets as widgets
from ipywidgets import Box, Layout
from IPython.display import clear_output

import os
import csv

print("Librerías cargadas")

Librerías cargadas


In [3]:
#@title Acceder al Drive

# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')

# directorio local en Google Drive
path = '/content/gdrive/My Drive/IA/demoML/texto/'  #@param {type:"string"}


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
#@title Cargar datos

#@markdown ### Archivo de datos a utilizar:
archivo_datos = 'FilmAffinity.csv'  #@param {type:"string"}
#@markdown ### Configuración del archivo CSV:
delimitador_columnas = '\\|\\|' #@param {type:"string"}

## selección de los parámetros 
def checkValidCharacter(str):
  if len(str) == 0:
    return False  
  for ch in str:
    # controla cada caracter
    if (ch != "\n") and \
       (unicodedata.category(ch) in {'Cc', 'Cf', 'Cn', 'Co', 'Cs'}):
      return False
    # nota: ver categorias en https://www.fileformat.info/info/unicode/category/index.htm
  return True    

def limpiarTexto(texto_ori):
  auxList = []
  t = list( texto_ori.strip() )
  for c in t:
    if checkValidCharacter(c):
      auxList.append( c )
  return "".join(auxList)


# función para cargar configuración datos automática
def cargarNombreClases(path, archivo_datos):
  # importa definición de la clase
  arClasesFN = archivo_datos.split('.')[0] + '_nombreClases.txt'
  if os.path.isfile( path + '/' + arClasesFN ):
    with open( path + '/' + arClasesFN, mode='r') as csvfile:
        r = csv.reader(csvfile, delimiter=',')
        auxAtributo = r.__next__()
        auxClases = r.__next__()
    print('\n> Definición de los valores discretos para la clase cargada de ' + arClasesFN +'.\n')
    return auxAtributo[0], ','.join(auxClases)
  else:
    return "", ""


# configura para que muestre todas las columnas y filas
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# inicializa valores
nombre_atributos_entrada = []
texto_cargado = None

# determina tipo de archivo a usar (si es de Texto o Datos)
if ".csv" in archivo_datos.lower():
    # se condidera un archivo de datos
    esArchivoDatos = True

    # función auxiliara para que no ejecute UI cada vez
    def hacerNada():
      return

    # se define esta función para que se ocupe de aplicar la configuración
    def on_buttonAplicar_clicked(b):  
      print("")
      funcionCambiaSeleccion_ConfigDatos(combo_att_clase.value)

    # aplica configuración de datos
    def funcionCambiaSeleccion_ConfigDatos(attClase):
      global texto_cargado

      if (attClase is None) or (attClase ==""):
        return
    
      # muestra resultados
      print("\n> Atributo texto: ", attClase)


      # genera los datos solo con los atributos seleccionados
      # se elimina ejemplos duplicados
      ndf = df[attClase]
      ndf = ndf.drop_duplicates(keep="first")
      # une todo con salto de lineas
      texto_cargado = "\n".join( np.array(ndf) )
      
      # saca caracteres en blanco al principio y fin
      # y caracteres de control especiales
      texto_cargado = limpiarTexto(texto_cargado)

      print("\n> Características: ")
      print (' -- Tamaño total del texto: {} caracteres'.format(len(texto_cargado)))

      # muestra los primeros 250 caracteres del texto
      print("\n -- Ejemplo: \n", texto_cargado[:250])


    ## aplicación de los parámetros elegidos

    # Carga los datos del CSV y muestra los primeros
    df = pd.read_csv(path + archivo_datos,  sep=delimitador_columnas, engine="python")
    print("Archivo de datos ", archivo_datos, " cargado")

    print("\n> Cabecera: ")
    print(df.head())
    print("\n> Características: ")
    print(df.describe())
    print("\n")

    # intenta cargar configuración asociada a los datos
    # trata de obtener la configuración del archivo asociado
    atributo_clase, nombre_clases = cargarNombreClases(path, archivo_datos)

    # muestra interface para cargar configuración

    # auxiliar para que muestre bien la descripción
    style_3D = {'description_width': 'initial'}

    tit = widgets.Label("Ajuste para configuración de los Datos: ")
        
    # prepara combo para determinar atributo clase
    selecc_atributos = [ ]
    selecc_atributos.extend( df.columns.values.tolist() )
    if (atributo_clase is None) or (atributo_clase=="") or (atributo_clase not in selecc_atributos):
      att_selecc_defecto = 0
    else:
      att_selecc_defecto = selecc_atributos.index(atributo_clase)
    combo_att_clase = widgets.Dropdown(
        options = selecc_atributos,
        value = selecc_atributos[att_selecc_defecto], # mostrar por defecto de config
        description = 'Atributo texto:',
        style=style_3D,
        disabled = False,
    )

    # prepara botón y grilla con objetos
    btnAplicar = widgets.Button(
        description='Aplicar'
    )
    configDatos_ui = widgets.GridBox(
          children=[tit, combo_att_clase, btnAplicar],
          layout=Layout(width='100%')  )
    btnAplicar.on_click(on_buttonAplicar_clicked)

    # clear_output()  
    out_config = widgets.interactive_output(hacerNada, {})  
    display(configDatos_ui)

    # ejecuta para que muestre
    on_buttonAplicar_clicked(btnAplicar)

else:
    # se condidera un archivo de texto
    esArchivoDatos = False

    # levanta el archivo de texto del Drive para procesar
    texto_cargado = open("".join(path + archivo_datos), 'rb').read().decode(encoding='utf-8', errors='ignore')
    print("Archivo de texto ", archivo_datos, " cargado")

    # saca caracteres en blanco al principio y fin
    # y caracteres de control especiales
    print("> Se limpian caracteres especiales de control")
    texto_cargado = limpiarTexto(texto_cargado)

    print("\n> Características: ")
    print (' -- Tamaño total del texto: {} caracteres'.format(len(texto_cargado)))

    # muestra los primeros 250 caracteres del texto
    print("\n -- Ejemplo: \n", texto_cargado[:250])

Archivo de datos  FilmAffinity.csv  cargado

> Cabecera: 
               film_name   gender  film_avg_rate  review_rate  \
0  Ocho apellidos vascos  Comedia            6.0            3   
1  Ocho apellidos vascos  Comedia            6.0            2   
2  Ocho apellidos vascos  Comedia            6.0            2   
3  Ocho apellidos vascos  Comedia            6.0            2   
4  Ocho apellidos vascos  Comedia            6.0            2   

                                        review_title  \
0     OCHO APELLIDOS VASCOS...Y NINGÚN NOMBRE PROPIO   
1                                     El perro verde   
2  Si no eres de comer mierda... no te comas esta...   
3                                    Aida: The movie   
4               UN HOMBRE SOLO (Julio Iglesias 1987)   

                                         review_text  
0  La mayor virtud de esta película es su existen...  
1  No soy un experto cinéfilo, pero pocas veces m...  
2  Si no eres un incondicional del humor estilo T

GridBox(children=(Label(value='Ajuste para configuración de los Datos: '), Dropdown(description='Atributo text…



> Atributo texto:  film_name

> Características: 
 -- Tamaño total del texto: 802 caracteres

 -- Ejemplo: 
 Ocho apellidos vascos
Lo imposible
Ocho apellidos catalanes
El orfanato
La gran aventura de Mortadelo y Filemón
Torrente 2: Misión en Marbella
Ágora
Perfectos desconocidos
Mar adentro
Torrente 4
Campeones
Las aventuras de Tadeo Jones
Torrente 3
Tadeo


> Atributo texto:  review_text

> Características: 
 -- Tamaño total del texto: 10756541 caracteres

 -- Ejemplo: 
 La mayor virtud de esta película es su existencia.El hecho de que podamos jugar con los tópicos más extremos de las identidades patrias (la andaluza y la vasca) sin que nadie se escandalice ni ponga el grito en el cielo, indica mucho de nuestra madur


In [5]:
#@title Preparar texto para Markovify

#utility function for text cleaning
def text_cleaner(text):
  # reemplaza salto de lineas y tabas por "." 
  # porque sino falla al procesar modelo
  text = text.replace("\n", ".")
  text = text.replace("\t", ".")
  # pone en minusculas
  text = text.lower()
  # otros reemplazos
  text = re.sub(r'--', ' ', text)
  text = re.sub('[\[].*?[\]]', '', text)
  text = re.sub(r'(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b','', text)
  # hace que se unan por "." con espacio atrás 
  # lo hace así para unir también frases que estén como vector
  text = ". ".join(text.split("."))
  return text

#apply cleaning function to corpus
texto = text_cleaner(texto_cargado)

print("\nEjemplo:")
print(texto[:250])


Ejemplo:
la mayor virtud de esta película es su existencia. el hecho de que podamos jugar con los tópicos más extremos de las identidades patrias (la andaluza y la vasca) sin que nadie se escandalice ni ponga el grito en el cielo, indica mucho de nuestra madu


In [6]:
#@title Establecer y entrenar cadena de Markov

# genera y compila el modelo
markovModel = markovify.Text(texto)
markovModel.compile(inplace = True)

print("Cadena de Markov para texto definida.")

Cadena de Markov para texto definida.


In [8]:
#@title Generar Texto

cant_frases_generar = 26 #@param {type:"slider", min:1, max:1000, step:5}
cant_maxima_caracteres = -1 #@param {type:"slider", min:-1, max:1000, step:1}
cant_tries = 500

# Print  randomly-generated sentences
print("> Frases generadas: ")
for i in range(cant_frases_generar):
  if cant_maxima_caracteres > 0:
    print("#"+str(i+1)+": ", markovModel.make_short_sentence(cant_maxima_caracteres, tries=cant_tries))
  else:
    print("#"+str(i+1)+": ", markovModel.make_sentence(tries=cant_tries))
  print("")


> Frases generadas: 
#1:  la película pero sin hacer un musical sin música original.

#2:  se notan los cortes continuos que produce el efecto contrario.

#3:  . . mejor no saber juzgar ni diferenciar aquellos aspectos que me hace mucha gracia el concepto de ritmo tan inadecuado y por desgracia es posible recuperarse, estás asombrado por las más de que el film nos cuenta algo que no deja de tener que recurrir a los personajes; el final de la butaca pero pensé en su pasado, fantasmas que van desfilando.

#4:  para mi gusto, de su encuentro casual en la pregunta ¿está película es mala, sea de aquellos menos exigentes, pero acumula una larga escena de las sectas podría haberse alcanzado una película muy recomendable, el reparto tiene algún gag, y ji ji.

#5:  . llevaba mucho tiempo sin explicaciones, etc, etc. pero lo que hace.

#6:  me alegro haber descubierto, el mundo que vea las películas basadas en la imagen del país? por dios, que alguien les diga ese maldito olor a hollywood y con 